In [1]:
!pip install pykrige

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 909.7/909.7 kB 24.9 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from pykrige.ok3d import OrdinaryKriging3D

In [3]:
import pandas as pd
import numpy as np
import os
from pykrige.ok3d import OrdinaryKriging3D
from sklearn.preprocessing import MinMaxScaler
import shutil

def process_and_save(train_path, val_path, test_path, output_dir):
    print(f"Processing {train_path} and {val_path}")
    
    df_train = pd.read_csv(train_path)
    df_val = pd.read_csv(val_path)
    
    # Kết hợp dữ liệu để có nhiều điểm hơn cho việc nội suy
    df = pd.concat([df_train, df_val], ignore_index=True)
    
    # Chuẩn bị dữ liệu
    df.rename(columns={'datetime': 'DATETIME', 'value': 'AWS'}, inplace=True)
    if 'time' in df.columns:
        df.rename(columns={'time': 'DATETIME'}, inplace=True)
    
    # Đảm bảo tên cột theo đúng yêu cầu
    if 'row' in df.columns and 'col' in df.columns:
        df.rename(columns={'row': 'ROW', 'col': 'COL'}, inplace=True)
    
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])
    df['AWS_MISSING'] = df['AWS'].isna() | (df['AWS'] < 0)
    df.loc[df['AWS'] < 0, 'AWS'] = np.nan
    
    # Chuyển đổi thời gian thành giá trị số để sử dụng trong Kriging 3D
    df['time_numeric'] = (df['DATETIME'] - df['DATETIME'].min()).dt.total_seconds()
    
    # Scale các cột khác ngoài AWS, ROW, COL, DATETIME
    columns_to_scale = [col for col in df.columns if col not in ['AWS', 'ROW', 'COL', 'DATETIME', 'AWS_MISSING', 'time_numeric']]
    scalers = {}
    
    # Scale từng cột riêng biệt
    if columns_to_scale:
        for col in columns_to_scale:
            if col in df.columns:
                try:
                    scaler = MinMaxScaler()
                    # Reshape để đảm bảo 2D
                    data_2d = df[col].values.reshape(-1, 1)
                    df[col] = scaler.fit_transform(data_2d)
                    scalers[col] = scaler
                except Exception as e:
                    print(f"Không thể scale cột {col}: {e}")
    
    # Chia dữ liệu thành các điểm đã biết và điểm cần nội suy
    df_known = df[~df['AWS_MISSING']][['ROW', 'COL', 'time_numeric', 'AWS']]
    df_missing = df[df['AWS_MISSING']]
    
    print(f"Số điểm đã biết: {len(df_known)}, Số điểm cần nội suy: {len(df_missing)}")
    
    # Nếu có quá nhiều điểm, có thể cần phải lấy mẫu để tăng hiệu suất
    if len(df_known) > 400:
        df_known = df_known.sample(n=400, random_state=42)
        print(f"Lấy mẫu 400 điểm đã biết để tăng hiệu suất")
    
    # Nội suy bằng Ordinary Kriging 3D cho tất cả các điểm thiếu
    batch_size = 50
    num_batches = len(df_missing) // batch_size + 1
    kriging_results = []
    
    print(f"Thực hiện nội suy Kriging với {num_batches} batch...")
    
    for i in range(num_batches):
        batch = df_missing.iloc[i * batch_size:(i + 1) * batch_size]
        if batch.empty:
            continue
            
        try:
            # Khởi tạo và thực hiện Ordinary Kriging 3D
            OK3D = OrdinaryKriging3D(
                df_known['ROW'].values,
                df_known['COL'].values,
                df_known['time_numeric'].values,
                df_known['AWS'].values,
                variogram_model="linear",
                verbose=False,
                enable_plotting=False
            )
            
            # Thực hiện nội suy
            z_pred, _ = OK3D.execute("points", 
                                     batch['ROW'].values, 
                                     batch['COL'].values, 
                                     batch['time_numeric'].values)
            
            # Lưu kết quả
            for idx, (index, row) in enumerate(batch.iterrows()):
                if z_pred[idx] < 0:
                    z_pred[idx] = 0
                kriging_results.append({
                    'index': index,
                    'AWS_kriging': z_pred[idx]
                })
                
            print(f"Đã hoàn thành batch {i+1}/{num_batches}")
        except Exception as e:
            print(f"Lỗi ở batch {i+1}: {e}")
    
    # Áp dụng kết quả nội suy trực tiếp vào cột AWS
    for result in kriging_results:
        df.loc[result['index'], 'AWS'] = result['AWS_kriging']
    
    # Clipping để xử lý outlier, chặn trên là 10
    df['AWS'] = np.clip(df['AWS'], None, 10)
    
    # Loại bỏ các cột tạm thời
    df = df.drop(columns=['time_numeric'] + 
                 [col for col in df.columns if col.startswith('AWS_') and col not in ['AWS', 'AWS_MISSING']])
    
    # Chia lại thành tập train và validation
    df_train_processed = df.iloc[:len(df_train)]
    df_val_processed = df.iloc[len(df_train):]
    
    # Xử lý tập test nếu có
    if test_path and os.path.exists(test_path):
        try:
            df_test = pd.read_csv(test_path)
            
            # Chuẩn bị dữ liệu test
            df_test.rename(columns={'datetime': 'DATETIME', 'value': 'AWS'}, inplace=True)
            if 'time' in df_test.columns:
                df_test.rename(columns={'time': 'DATETIME'}, inplace=True)
            
            if 'row' in df_test.columns and 'col' in df_test.columns:
                df_test.rename(columns={'row': 'ROW', 'col': 'COL'}, inplace=True)
            
            # Áp dụng clipping cho AWS trong test set
            if 'AWS' in df_test.columns:
                df_test['AWS'] = np.clip(df_test['AWS'], None, 10)
            
            # Scale các cột trong test set
            for col in df_test.columns:
                if col in scalers and col not in ['AWS', 'ROW', 'COL', 'DATETIME']:
                    try:
                        # Scale cột sử dụng scaler tương ứng đã được fit từ tập train/val
                        data_2d = df_test[col].values.reshape(-1, 1)
                        df_test[col] = scalers[col].transform(data_2d)
                    except Exception as e:
                        print(f"Không thể scale cột {col} trong test set: {e}")
            
            # Lưu test set
            os.makedirs(output_dir, exist_ok=True)
            df_test.to_csv(os.path.join(output_dir, "merged_test.csv"), index=False)
            print(f"Processed test file saved to {os.path.join(output_dir, 'merged_test.csv')}")
        except Exception as e:
            print(f"Lỗi khi xử lý file test: {e}")
            # Sao chép file test gốc nếu không thể xử lý
            if os.path.exists(test_path):
                os.makedirs(output_dir, exist_ok=True)
                shutil.copy(test_path, os.path.join(output_dir, "merged_test.csv"))
                print(f"Đã sao chép file test gốc vào {os.path.join(output_dir, 'merged_test.csv')}")
    
    # Lưu kết quả
    os.makedirs(output_dir, exist_ok=True)
    df_train_processed.to_csv(os.path.join(output_dir, "processed_train.csv"), index=False)
    df_val_processed.to_csv(os.path.join(output_dir, "processed_val.csv"), index=False)
    
    print(f"Processed train saved to {os.path.join(output_dir, 'processed_train.csv')}")
    print(f"Processed val saved to {os.path.join(output_dir, 'processed_val.csv')}")
    
    return df_train_processed, df_val_processed

def process_all_folds():
    months = ["2019-04", "2019-10", "2020-04", "2020-10"]
    
    for month in months:
        print(f"\n{'='*50}")
        print(f"Processing month: {month}")
        print(f"{'='*50}")
        
        for fold in range(1, 6):  # Xử lý fold_1 đến fold_5
            base_root = f"/kaggle/input/dataset-k-fold/{month}/fold_{fold}"
            output_root = f"/kaggle/working/{month}/fold_{fold}"
            
            train_path = os.path.join(base_root, "merged_train.csv")
            val_path = os.path.join(base_root, "merged_val.csv")
            test_path = os.path.join(base_root, "merged_test.csv")
            
            if os.path.exists(train_path) and os.path.exists(val_path):
                print(f"\nProcessing {month} - fold_{fold}")
                process_and_save(train_path, val_path, test_path, output_root)
            else:
                print(f"Skipping {month} - fold_{fold} (files not found)")

# Chạy xử lý cho tất cả các fold trong tất cả các tháng
if __name__ == "__main__":
    process_all_folds()


Processing month: 2019-04

Processing 2019-04 - fold_1
Processing /kaggle/input/dataset-k-fold/2019-04/fold_1/merged_train.csv and /kaggle/input/dataset-k-fold/2019-04/fold_1/merged_val.csv
Số điểm đã biết: 43823, Số điểm cần nội suy: 20305
Lấy mẫu 400 điểm đã biết để tăng hiệu suất
Thực hiện nội suy Kriging với 407 batch...
Đã hoàn thành batch 1/407
Đã hoàn thành batch 2/407
Đã hoàn thành batch 3/407
Đã hoàn thành batch 4/407
Đã hoàn thành batch 5/407
Đã hoàn thành batch 6/407
Đã hoàn thành batch 7/407
Đã hoàn thành batch 8/407
Đã hoàn thành batch 9/407
Đã hoàn thành batch 10/407
Đã hoàn thành batch 11/407
Đã hoàn thành batch 12/407
Đã hoàn thành batch 13/407
Đã hoàn thành batch 14/407
Đã hoàn thành batch 15/407
Đã hoàn thành batch 16/407
Đã hoàn thành batch 17/407
Đã hoàn thành batch 18/407
Đã hoàn thành batch 19/407
Đã hoàn thành batch 20/407
Đã hoàn thành batch 21/407
Đã hoàn thành batch 22/407
Đã hoàn thành batch 23/407
Đã hoàn thành batch 24/407
Đã hoàn thành batch 25/407
Đã hoà

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


Số điểm đã biết: 36687, Số điểm cần nội suy: 29553
Lấy mẫu 400 điểm đã biết để tăng hiệu suất
Thực hiện nội suy Kriging với 592 batch...
Đã hoàn thành batch 1/592
Đã hoàn thành batch 2/592
Đã hoàn thành batch 3/592
Đã hoàn thành batch 4/592
Đã hoàn thành batch 5/592
Đã hoàn thành batch 6/592
Đã hoàn thành batch 7/592
Đã hoàn thành batch 8/592
Đã hoàn thành batch 9/592
Đã hoàn thành batch 10/592
Đã hoàn thành batch 11/592
Đã hoàn thành batch 12/592
Đã hoàn thành batch 13/592
Đã hoàn thành batch 14/592
Đã hoàn thành batch 15/592
Đã hoàn thành batch 16/592
Đã hoàn thành batch 17/592
Đã hoàn thành batch 18/592
Đã hoàn thành batch 19/592
Đã hoàn thành batch 20/592
Đã hoàn thành batch 21/592
Đã hoàn thành batch 22/592
Đã hoàn thành batch 23/592
Đã hoàn thành batch 24/592
Đã hoàn thành batch 25/592
Đã hoàn thành batch 26/592
Đã hoàn thành batch 27/592
Đã hoàn thành batch 28/592
Đã hoàn thành batch 29/592
Đã hoàn thành batch 30/592
Đã hoàn thành batch 31/592
Đã hoàn thành batch 32/592
Đã hoàn 

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


Processed test file saved to /kaggle/working/2019-10/fold_1/merged_test.csv
Processed train saved to /kaggle/working/2019-10/fold_1/processed_train.csv
Processed val saved to /kaggle/working/2019-10/fold_1/processed_val.csv

Processing 2019-10 - fold_2
Processing /kaggle/input/dataset-k-fold/2019-10/fold_2/merged_train.csv and /kaggle/input/dataset-k-fold/2019-10/fold_2/merged_val.csv


...

Processed test file saved to /kaggle/working/2020-10/fold_4/merged_test.csv
Processed train saved to /kaggle/working/2020-10/fold_4/processed_train.csv
Processed val saved to /kaggle/working/2020-10/fold_4/processed_val.csv

Processing 2020-10 - fold_5
Processing /kaggle/input/dataset-k-fold/2020-10/fold_5/merged_train.csv and /kaggle/input/dataset-k-fold/2020-10/fold_5/merged_val.csv


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


Số điểm đã biết: 171178, Số điểm cần nội suy: 27542
Lấy mẫu 400 điểm đã biết để tăng hiệu suất
Thực hiện nội suy Kriging với 551 batch...
Đã hoàn thành batch 1/551
Đã hoàn thành batch 2/551
Đã hoàn thành batch 3/551
Đã hoàn thành batch 4/551
Đã hoàn thành batch 5/551
Đã hoàn thành batch 6/551
Đã hoàn thành batch 7/551
Đã hoàn thành batch 8/551
Đã hoàn thành batch 9/551
Đã hoàn thành batch 10/551
Đã hoàn thành batch 11/551
Đã hoàn thành batch 12/551
Đã hoàn thành batch 13/551
Đã hoàn thành batch 14/551
Đã hoàn thành batch 15/551
Đã hoàn thành batch 16/551
Đã hoàn thành batch 17/551
Đã hoàn thành batch 18/551
Đã hoàn thành batch 19/551
Đã hoàn thành batch 20/551
Đã hoàn thành batch 21/551
Đã hoàn thành batch 22/551
Đã hoàn thành batch 23/551
Đã hoàn thành batch 24/551
Đã hoàn thành batch 25/551
Đã hoàn thành batch 26/551
Đã hoàn thành batch 27/551
Đã hoàn thành batch 28/551
Đã hoàn thành batch 29/551
Đã hoàn thành batch 30/551
Đã hoàn thành batch 31/551
Đã hoàn thành batch 32/551
Đã hoàn

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)


Processed test file saved to /kaggle/working/2020-10/fold_5/merged_test.csv
Processed train saved to /kaggle/working/2020-10/fold_5/processed_train.csv
Processed val saved to /kaggle/working/2020-10/fold_5/processed_val.csv
